In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext sql

In [2]:
%sql sqlite:///results.db

# Overriding Defaults

In [24]:
%%sql
with total as (select count(*) as total from alias)
select 
    count(distinct alias_id) as '#', 
    round(count(distinct alias_id)*100.0/total,2) as '%' 
from alias join command using (alias_id)
join total
where alias.name = command.name;

 * sqlite:///results.db
Done.


#,%
319239,14.48


### Top Redefined Commands

In [25]:
%%sql
with
    total as (select count(*) as total from alias),
    total_cmd as (select name as cmd_name, count(distinct alias_id) as total_cmd from command group by name)
select 
    command.name, 
    count(distinct alias_id) as num_overrides,
    round(count(distinct alias_id)*100.0/total,2) as per_of_aliases,
    round(count(distinct alias_id)*100.0/total_cmd,2) as per_of_command
from alias join command using (alias_id)
join total_cmd on command.name = total_cmd.cmd_name
join total
where alias.name = command.name
group by command.name
order by num_overrides desc
limit 50;

 * sqlite:///results.db
Done.


name,num_overrides,per_of_aliases,per_of_command
ls,75374,3.42,28.99
grep,42662,1.94,50.6
egrep,27515,1.25,89.17
fgrep,26793,1.22,99.19
cp,13853,0.63,80.09
rm,12921,0.59,55.37
mv,12101,0.55,89.3
df,8427,0.38,86.73
mkdir,5975,0.27,64.48
du,5672,0.26,47.42


In [22]:
%%sql
with
    total as (select count(*) as total from alias),
    total_cmd as (select count(distinct alias_id) as total_cmd from command where name in ('grep','fgrep','egrep'))
select 
    count(distinct alias_id) as num_overrides,
    round(count(distinct alias_id)*100.0/total,2) as per_of_aliases,
    round(count(distinct alias_id)*100.0/total_cmd,2) as per_of_command
from alias join command using (alias_id)
join total_cmd
join total
where alias.name = command.name
and alias.name in ('grep','fgrep','egrep')

 * sqlite:///results.db
Done.


num_overrides,per_of_aliases,per_of_command
96970,4.4,68.27


In [31]:
%%sql
with
    total_cmd_overrides as (
        select command.name as cmd_name, count(distinct alias_id) as total_cmd_overrides 
        from alias join command using (alias_id)
        where alias.name = command.name
        group by command.name
    )
select 
    command.name, command.arguments,
    count(distinct alias_id) as num_overrides_with_args,
    round(count(distinct alias_id)*100.0/total_cmd_overrides,2) as per_of_command_overrides
from alias join command using (alias_id)
join total_cmd_overrides on command.name = total_cmd_overrides.cmd_name
where alias.name = command.name
group by command.arguments, command.name
order by num_overrides_with_args desc
limit 50;

 * sqlite:///results.db
Done.


name,arguments,num_overrides_with_args,per_of_command_overrides
ls,--color=auto,37230,49.39
grep,--color=auto,33198,77.82
egrep,--color=auto,25486,92.63
fgrep,--color=auto,25112,93.73
cp,-i,9189,66.33
rm,-i,9087,70.33
mv,-i,8662,71.58
ls,-G,7240,9.61
df,-h,6173,73.25
sudo,,4231,100.0
